In [3]:
#import required functions
import warnings
warnings.filterwarnings("ignore")
import math
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pylab as plt
import numpy as np
from IPython.display import Image
import matplotlib as matplotlib
import datetime
from pandas import DataFrame
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
from boto.mws.connection import MWSConnection
import mws
from bs4 import BeautifulSoup
import mechanize
import urllib.request
import http.cookiejar as cookielib
import requests
from pprint import pprint

In [5]:
filename =  '/Users/JoyThompson/Desktop/ddasins.xlsx'
asinpd = pd.read_excel(filename, sheetname='asins')
asinpd=asinpd.set_index('(Child) ASIN')
dfasin=pd.DataFrame(index=asinpd.index, columns=['Toys & Games (See Top 100', 'Preschool Dolls & Dollhouses', 'Dolls'])

In [ ]:
agentlist = [ 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)']

In [7]:
def getrankings(asin, headers, dfasin):   
    url='https://www.amazon.com/dp/' + asin+ '/'
    headers = {'authority': 'www.amazon.com', 'pragma': 'no-cache',  'cache-control': 'no-cache',    'dnt': '1', 'upgrade-insecure-requests': '1',     'user-agent': useragent,
        'sec-fetch-site': 'none', 'sec-fetch-mode': 'navigate', 'sec-fetch-dest': 'document',     'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8'}
    response =urllib.request.Request(url, headers=headers)
    result = urllib.request.urlopen(response)
    resulttext = result.read()
    soup = BeautifulSoup(resulttext)
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    newtext=text[float(text.find("Best Sellers Rank")+250):end]
    newend=newtext.split('Customer Reviews')[0]
    newend=newend.split("#")[1:]
    for ind in newend:
        rating=ind.split(' ')[0]
        cat=ind.split('in ')[1]
        cat=cat.split('\n')[0]
        if rating.find(',')!=-1:
            ratesplit=rating.split(',')
            ratesplit=ratesplit[0]+ratesplit[1]
            rating=float(ratesplit)
        else:
            rating=float(rating)

        dfasin.loc[asin, cat]=rating
    return dfasin

In [6]:
for asin in dfasin.index:
    dfasin=getrankings(asin, headers, dfasin)
filenamefinal= '/Users/JoyThompson/Desktop/AMAZONPRODSCRAPER.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
dfasin.to_excel(writer, sheet_name='SCRAPE')
writer.save()

Best Sellers Rank
#4,185 in Toys & Games (See Top 100 in Toys & Games)
#100 in Preschool Dolls & Dollhouses
#129 in Dolls

Toys & Games (See Top 100 
4
4185
Toys & Games (See Top 100 
Preschool Dolls & Dollhouses
Preschool Dolls & Dollhouses
Dolls
Dolls
Best Sellers Rank
#7,383 in Toys & Games (See Top 100 in Toys & Games)
#42 in Gags & Practical Joke Toys
#3,535 in Pre-Kindergarten Toys

Toys & Games (See Top 100 
7
7383
Toys & Games (See Top 100 
Gags & Practical Joke Toys
Gags & Practical Joke Toys
Pre-Kindergarten Toys
3
3535
Pre-Kindergarten Toys
Best Sellers Rank
#19,072 in Toys & Games (See Top 100 in Toys & Games)
#430 in Preschool Dolls & Dollhouses
#561 in Dolls

Toys & Games (See Top 100 
19
19072
Toys & Games (See Top 100 
Preschool Dolls & Dollhouses
Preschool Dolls & Dollhouses
Dolls
Dolls
Best Sellers Rank
#38,632 in Toys & Games (See Top 100 in Toys & Games)
#1,076 in Dolls

Toys & Games (See Top 100 
38
38632
Toys & Games (See Top 100 
Dolls
1
1076
Dolls
Best Sellers R